In [1]:
import pandas as pd
import glob, os
pd.set_option('float_format', '{:f}'.format)

dateparse = lambda x: pd.datetime.strptime(x, '%Y%m%d')

pd.options.display.max_columns = 999
pd.options.display.max_rows = 200

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings('ignore')

In [2]:
path = r'C:\Users\parsh\Desktop\Exchange\data\US_stocks_csv'
file_name = '/NYSE/ABX.csv'
file_ticker = path + file_name

main_df = pd.read_csv(file_ticker)
main_df['Date'] = pd.to_datetime(main_df['Date'], format='%Y%m%d', errors='ignore')
main_df['Date_2'] = main_df['Date'].dt.strftime('%d-%m-%Y')

In [3]:
def addMACD(df):
    ema_fast = df['Close'].ewm(span=12).mean()
    ema_slow = df['Close'].ewm(span=26).mean()
    signal_line = df['Close'].ewm(span=9).mean()
    df['macd'] = ema_fast - ema_slow
    df['macd_signal'] = df.macd.ewm(span=9, adjust=False).mean()
    df['macdh'] = df['macd'] - df['macd_signal']
    return df

test_asset = addMACD(main_df)

In [4]:
coordinate_dict = {}


# Задаём начальные значения как затычки для первой итерации 
# TODO пофиксить этот костыль

previous_value = 0.00001

coordinate_up_break = 0.00001
    
coordinate_down_break = 0.00001

for ind, row in test_asset.iterrows():
    
    current_macd = row['macdh']
    
#     вычисляем индекс перехода от минусового значения MACD к плюсовому, то есть пересечение нуля
    if (current_macd < 0) & (previous_value > 0):
        coordinate_up_break = ind
        
#     вычисляем индекс перехода от плюсового значения MACD к минусовому, то есть пересечение нуля        
    if (current_macd > 0) & (previous_value < 0):
        coordinate_down_break = ind -1
        coordinate_dict[coordinate_up_break] = coordinate_down_break
        
    previous_value = current_macd  

In [5]:
# Проверяем наши координаты
# внутри словаря - ключ : начало отрицательной гистограммы (предыдущее значение MACD - положительное)
# значение: конец отрицательной гистограммы (следующее значение MACD - положительное)
for state, capital in coordinate_dict.items(): 
    print(state, ":", capital)

12 : 28
38 : 39
41 : 52
56 : 70
127 : 134
146 : 160
176 : 192
194 : 194
196 : 204
216 : 220
252 : 277
317 : 338
344 : 354
356 : 356
358 : 374
389 : 391
405 : 421
431 : 448
452 : 465
513 : 521
537 : 539
548 : 568
582 : 607
626 : 626
640 : 652
657 : 658
671 : 701
729 : 743
745 : 764
787 : 808
813 : 832
857 : 885
888 : 898
910 : 918
940 : 948
957 : 986
1013 : 1016
1019 : 1019
1022 : 1024
1039 : 1054
1068 : 1079
1088 : 1092
1130 : 1169
1187 : 1190
1218 : 1227
1229 : 1245
1255 : 1260
1262 : 1264
1274 : 1274
1279 : 1291
1299 : 1327
1347 : 1359
1390 : 1398
1405 : 1424
1431 : 1449
1464 : 1468
1500 : 1519
1529 : 1534
1547 : 1556
1564 : 1584
1612 : 1618
1630 : 1649
1657 : 1677
1708 : 1716
1723 : 1730
1736 : 1745
1751 : 1751
1763 : 1771
1778 : 1808
1815 : 1815
1821 : 1831
1870 : 1872
1877 : 1881
1886 : 1899
1901 : 1910
1931 : 1949
1954 : 1959
1965 : 1978
2003 : 2017
2045 : 2060
2097 : 2119
2132 : 2143
2150 : 2163
2165 : 2166
2169 : 2181
2212 : 2213
2215 : 2218
2225 : 2239
2253 : 2253
2261 : 2261


In [6]:
test_asset['MACD_signal'] = False

skip_first_iterate = True

previous_peak = 0

# Итерируемся по словарю с координатами
for begin_hist, end_hist in coordinate_dict.items(): 
    
#     пропускаем первую итерацию, так как нам для рассчетов нужно текущее и предыдущее значение
    if skip_first_iterate == True:
        mask = (test_asset.index >= begin_hist) & (test_asset.index <= end_hist)
        previous_peak = test_asset['macdh'].iloc[mask].min()
        skip_first_iterate = False
        continue
    
#     вырезаем гистограмму по координатам и находим её минимум
    mask = (test_asset.index >= begin_hist) & (test_asset.index <= end_hist)
    current_peak = test_asset['macdh'].iloc[mask].min()
    
  # Если мы находимся в последнем дне и в нём найден минимум - значит отката не произошло и условия дивергенции не выполняются. Выходим из цикла
    if current_peak == test_asset["macdh"].iloc[-1]:
        break
    
#     фильтруем микродивегенции
    if previous_peak * 0.75 < current_peak:
        test_asset['MACD_signal'].iloc[mask]=True
        
    print(begin_hist, ":", end_hist,  " previous_peak : ", previous_peak, "current_peak :", current_peak)     
    
    previous_peak = current_peak

38 : 39  previous_peak :  -0.008068349059326766 current_peak : -0.0022374750375986265
41 : 52  previous_peak :  -0.0022374750375986265 current_peak : -0.003425476124293981
56 : 70  previous_peak :  -0.003425476124293981 current_peak : -0.001240342463455502
127 : 134  previous_peak :  -0.001240342463455502 current_peak : -0.006468120242281158
146 : 160  previous_peak :  -0.006468120242281158 current_peak : -0.023378531021782396
176 : 192  previous_peak :  -0.023378531021782396 current_peak : -0.022153972943031723
194 : 194  previous_peak :  -0.022153972943031723 current_peak : -0.001230077078795782
196 : 204  previous_peak :  -0.001230077078795782 current_peak : -0.0026656037851343455
216 : 220  previous_peak :  -0.0026656037851343455 current_peak : -0.005102011134025596
252 : 277  previous_peak :  -0.005102011134025596 current_peak : -0.008149647036867866
317 : 338  previous_peak :  -0.008149647036867866 current_peak : -0.016042547678563973
344 : 354  previous_peak :  -0.01604254767856

2508 : 2511  previous_peak :  -0.2886072722659526 current_peak : -0.0989837194757966
2522 : 2532  previous_peak :  -0.0989837194757966 current_peak : -0.1862162580439432
2548 : 2548  previous_peak :  -0.1862162580439432 current_peak : -0.02307713408644553
2575 : 2600  previous_peak :  -0.02307713408644553 current_peak : -0.2598763611790513
2627 : 2636  previous_peak :  -0.2598763611790513 current_peak : -0.16981990817214598
2644 : 2661  previous_peak :  -0.16981990817214598 current_peak : -0.22045512001122752
2673 : 2677  previous_peak :  -0.22045512001122752 current_peak : -0.06705712144749881
2686 : 2699  previous_peak :  -0.06705712144749881 current_peak : -0.06734316634458062
2703 : 2718  previous_peak :  -0.06734316634458062 current_peak : -0.14315486445676223
2746 : 2758  previous_peak :  -0.14315486445676223 current_peak : -0.15691811456001553
2770 : 2771  previous_peak :  -0.15691811456001553 current_peak : -0.024212785156827565
2787 : 2815  previous_peak :  -0.0242127851568275

5044 : 5051  previous_peak :  -0.23341520821148515 current_peak : -0.028291404043729873
5075 : 5107  previous_peak :  -0.028291404043729873 current_peak : -0.3273573370294465
5116 : 5122  previous_peak :  -0.3273573370294465 current_peak : -0.08433376417671967
5152 : 5170  previous_peak :  -0.08433376417671967 current_peak : -0.2020648258301458
5184 : 5193  previous_peak :  -0.2020648258301458 current_peak : -0.15730106392825316
5211 : 5211  previous_peak :  -0.15730106392825316 current_peak : -0.0023037792681009917
5214 : 5241  previous_peak :  -0.0023037792681009917 current_peak : -0.3683183205143502
5266 : 5274  previous_peak :  -0.3683183205143502 current_peak : -0.0780370914094074
5292 : 5295  previous_peak :  -0.0780370914094074 current_peak : -0.040860543756376266
5299 : 5328  previous_peak :  -0.040860543756376266 current_peak : -0.3315045612388765
5331 : 5332  previous_peak :  -0.3315045612388765 current_peak : -0.022128238048302995
5369 : 5395  previous_peak :  -0.02212823804

7793 : 7793  previous_peak :  -0.05202106363192594 current_peak : -0.005025024817297507
7803 : 7805  previous_peak :  -0.005025024817297507 current_peak : -0.029762789852259053
7834 : 7860  previous_peak :  -0.029762789852259053 current_peak : -0.2238737559134627
7881 : 7886  previous_peak :  -0.2238737559134627 current_peak : -0.05164631185641433
7888 : 7899  previous_peak :  -0.05164631185641433 current_peak : -0.3104527897731708
7912 : 7914  previous_peak :  -0.3104527897731708 current_peak : -0.059751807140117186
7925 : 7938  previous_peak :  -0.059751807140117186 current_peak : -0.31245005033495293
7944 : 7973  previous_peak :  -0.31245005033495293 current_peak : -0.43062511289441896
7984 : 7993  previous_peak :  -0.43062511289441896 current_peak : -0.15933559617481297
8011 : 8025  previous_peak :  -0.15933559617481297 current_peak : -0.28492113513488937
8035 : 8041  previous_peak :  -0.28492113513488937 current_peak : -0.08832280337141374
8076 : 8076  previous_peak :  -0.08832280

In [7]:
test_asset[['macdh','MACD_signal']].head(200)

,macdh,MACD_signal
0,0.000000,False
1,0.000000,False
2,0.000000,False
3,0.000000,False
4,0.000000,False
5,0.000000,False
6,0.000000,False
7,0.000000,False
8,0.000000,False
9,0.003217,False


# Turtle simple soup

In [8]:
extremum_window = 20 # баров для расчета колонки с n-барным экстремумом
col_min_name = 'min_'+str(extremum_window)+'_bars'

# число дней с текущего, когда сравниваем - пробили n-барный экстремум или нет
# не путать с extremum_window
break_window = 20 

# зона "баров назад" (шифта) от текущего, в которую не должен попасть n-дневный экструмум
# из колонки col_max_name. На самом деле получается зона = (empty_zone-1), 
# то есть в дне shift(empty_zone) экстремум еще может быть 
empty_zone = 4

signal_col = 'SOUP_' + str(extremum_window) + '_' + str(break_window) + '_' + str(empty_zone)

In [9]:
# !!!!!!!!!!! Важно!!!!!!!!!!!
# nan присваиваем минус -1e9 - потому что не для акций, а индексов могут быть отрицательные значения!!!
# Следовательно когда ищем минимумы - лучше присваивать +1e9 (цены акций из-за adjusted 
# могут быть сильно выше разумных величин)

test_asset[col_min_name] = test_asset['Low'][test_asset['Low'].rolling(extremum_window).min() == test_asset['Low']]
test_asset[col_min_name] = test_asset[col_min_name].fillna(1e9)

In [10]:
test_asset['min_Nmin'] = test_asset[col_min_name].shift(empty_zone).rolling(break_window-empty_zone+1).min()
test_asset['min_low_empty_zone'] = test_asset['Low'].shift(1).rolling(empty_zone-1).min()
test_asset['long_signal'] = ((test_asset['Low'] < test_asset['min_Nmin']) & \
                        (test_asset['min_Nmin'] < test_asset['min_low_empty_zone']) & \
                        (test_asset['Close'] > test_asset['min_Nmin'])) * (1)

# Signal comparing


In [11]:
test_asset['divergence_signal'] = (test_asset['long_signal'] == 1) & (test_asset['MACD_signal'])

In [12]:
# из-за обрезки датафрейма вверху ради красивого графика test_asset = test_asset[test_asset['Date'] > '2016-10-10'] сигналов мало
# если не обрезать, будет больше  :)
test_asset[test_asset['divergence_signal'] == True]

,Ticker,Date,Open,High,Low,Close,Volume,Date_2,macd,macd_signal,macdh,MACD_signal,min_20_bars,min_Nmin,min_low_empty_zone,long_signal,divergence_signal
455,ABX,1986-11-20,1.748550,1.764170,1.717330,1.748550,270223,20-11-1986,-0.014543,0.000323,-0.014866,True,1.717330,1.732940,1.748550,1,True
1089,ABX,1989-05-25,4.808520,4.870970,4.746080,4.839750,305452,25-05-1989,-0.080082,-0.069315,-0.010767,True,4.746080,4.808520,4.839750,1,True
3037,ABX,1997-02-06,26.500000,26.500000,25.750000,26.125000,1729100,06-02-1997,-0.297316,-0.288456,-0.008860,True,25.750000,26.000000,26.250000,1,True
3829,ABX,2000-03-29,16.187500,16.375000,15.562500,15.937500,2192300,29-03-2000,-0.192999,-0.161974,-0.031026,True,15.562500,15.875000,16.062500,1,True
5045,ABX,2005-02-01,21.810000,21.990000,21.610000,21.740000,1644800,01-02-2005,-0.330859,-0.306331,-0.024528,True,21.610000,21.720000,21.750000,1,True
5467,ABX,2006-10-04,28.640000,28.910000,27.630500,28.840000,9065000,04-10-2006,-0.449526,-0.444889,-0.004637,True,27.630500,27.920000,28.400000,1,True
5649,ABX,2007-06-27,28.060000,28.670000,27.790000,28.670000,5654500,27-06-2007,-0.085252,-0.084654,-0.000598,True,27.790000,27.970000,27.980000,1,True
7638,ABX,2015-05-21,12.420000,12.490000,12.235000,12.440000,8619981,21-05-2015,0.060338,0.161320,-0.100982,True,12.235000,12.270000,12.420000,1,True
7710,ABX,2015-09-02,6.740000,6.780000,6.500000,6.690000,16477137,02-09-2015,-0.375169,-0.361977,-0.013192,True,6.500000,6.510000,6.630000,1,True
8418,ABX,2018-06-26,12.790000,12.800000,12.570000,12.750000,9182437,26-06-2018,-0.092714,-0.075651,-0.017063,True,12.570000,12.700000,12.720000,1,True


In [13]:
def plot_signal(df, signal_name):
    # рисуем свечной график с объемами
    # signal_name - колонка с сигналом
    # ниже кода для трех состояний сигнала: +1, -1, 0
    # +1 синий, -1 розовый, 0 цвет свечек не меняется

    df['Date_2'] = df['Date'].dt.strftime('%d-%m-%Y')

    color_signal_long = '#0028FF' # цвет сигнала для лонга - синий
    color_signal_short = '#ff00f1' # цвет сигнала для шорта - розовый
    
    mask_long = df[signal_name]==True
        
    name_long = 'Long_' + str(round(df[mask_long].shape[0]*100/df.shape[0],1)) + '%'
        
    ticker_name = df.Ticker[0]
    data = [ dict(
        name = ticker_name + '_' + signal_name,
        type = 'candlestick',
        x = df['Date_2'],
        yaxis = 'y2',
        open = df['Open'],
        high = df['High'],
        low = df['Low'],
        close = df['Close'],
        opacity = 0.8
    ),
        dict(
        name = name_long,
        type = 'candlestick',
        x = df['Date_2'][mask_long],
        yaxis = 'y2',
        open = df['Open'][mask_long],
        high = df['High'][mask_long],
        low = df['Low'][mask_long],
        close = df['Close'][mask_long],
        hoverinfo='none',
        increasing = dict(line = dict(color = '#0028FF')),
        decreasing = dict(line = dict(color = '#ff00f1'))
    )
           ]

    layout=dict()

    fig = dict(data=data, layout=layout)

    fig['layout'] = dict()
    #fig['layout']['plot_bgcolor'] = 'rgb(250, 250, 250)'
    #categoryorder = 'category ascending', 
    fig['layout']['xaxis'] = dict(type ='category', 
                                  tickfont=dict(size=10),
                                  rangeselector = dict(visible = True), 
                                  rangeslider = dict(visible = False))
    fig['layout']['yaxis'] = dict(domain = [0, 0.2], 
                                  showticklabels = False, 
                                  autorange = True, 
                                  type = 'log',
#                                  title = 'Volume'
                                 )
    fig['layout']['yaxis2'] = dict(domain = [0.1, 0.95], 
                                   autorange = True, 
                                   type = 'log',
                                   title = ticker_name + ' prices')
    fig['layout']['legend'] = dict(orientation = 'h', 
                                   y = 0.95, x = 0.1, 
                                   yanchor = 'bottom')
    fig['layout']['margin'] = dict(t = 40, b = 40, r = 40, l = 40)
    
   


    plotly.offline.iplot(fig)
    return

In [20]:
def plot_df(df, signal_name):
    
    mask_long = df[signal_name]==True

    
    candle_plot =  dict(
        type = 'candlestick',
        open = test_asset.Open,
        high = test_asset.High,
        low = test_asset.Low,
        close = test_asset.Close,
        x = test_asset.Date_2,
        yaxis = 'y',
        name = 'ABX'
    )
    
    candle_plot_blue =  dict(
        type = 'candlestick',
        open = test_asset.Open[mask_long],
        high = test_asset.High[mask_long],
        low = test_asset.Low[mask_long],
        close = test_asset.Close[mask_long],
        x = test_asset.Date_2[mask_long],
        yaxis = 'y',
        name = 'ABX',
        hoverinfo='none',
        decreasing = dict(line = dict(color = 'blue'))
    )
    
    macd_signal =  dict( x=test_asset.Date_2, 
                         y=test_asset.macd_signal,    
                        marker={"color": "green"},
                         name='macd_signal',
                         type='scatter',
                         
                         yaxis='y'
                          )
    
    macd =  dict( x=test_asset.Date_2,
                     y=test_asset.macd,   
         
                     marker={"color": "blue"},
                     name='macd',
                     type='scatter',

                     yaxis='y'
                      )
    
    
    macdh = dict( x=test_asset.Date_2, 
                         y=test_asset.macdh,   
         
                         fill= 'tozeroy',
                         marker={"color": "grey"},
                         name='macdh',
                         type='scatter',
                         
                         yaxis='y'
                          )
    
    data = [candle_plot, candle_plot_blue, macd_signal, macd, macdh]
    
    
    layout = {
      "height": 800, 
      "yaxis": {"domain": [0.5, 1]},
      "yaxis2": {"domain": [0, 0.5]}
    }

    fig = dict( data=data, layout=layout )
    
    
    fig['layout']['xaxis'] = dict(type ='category', 
                              tickfont=dict(size=10),
                              rangeselector = dict(visible = True), 
                              rangeslider = dict(visible = False)
                                 )
    
    
    fig['layout']['legend'] = dict( orientation = 'h', y=0.9, x=0.3, yanchor='bottom' )
    fig['layout']['margin'] = dict( t=40, b=40, r=40, l=40 )
    
    fig['layout']['legend'] = dict(orientation = 'h', 
                                   y = 0.95, x = 0.1, 
                                   yanchor = 'bottom')

    plotly.offline.iplot( fig, validate = False )

In [15]:
# для того, чтобы графики MACD было видно
test_asset['macdh'] *= 10
test_asset['macd'] *= 10
test_asset['macd_signal'] *= 10

In [21]:
plot_df(test_asset, signal_name='divergence_signal')